In [1]:
import json
import pickle

import scipy

import numpy as np
import pandas as pd



In [49]:
def read_in_csv():
    data = pd.read_csv("../data/data.csv")
    data.rename(columns={0: 'Date'}, inplace=False)
    cols = data.columns.tolist()
    cols = cols[-1:] + cols[1:-1]
    data = data[cols]
    data["Date"] = pd.to_datetime(data["Date"])
    data = data.set_index("Date")
    data.index = data.index - pd.Timedelta(days=1)

    return data


In [50]:
def consecutive_nans(row):
    current_consec = 0
    location_of_nans = {}

    for key, value in row.items():
        if pd.isna(value):

            if current_consec == 0:
                start_date = key
                location_of_nans[key] = pd.Timedelta(1, 'd')
            else:
                location_of_nans[start_date] += pd.Timedelta(1, 'd')
            current_consec += 1
        else:
            current_consec = 0

    return location_of_nans


In [51]:
def calculate_missing_values(df, to_fill, missing_data):
    data_nans_removed = df
    for station in to_fill.index:
        usable_indices = to_fill.loc[station, 'usable']
        x_index = to_fill.loc[station, 'x']
        x = df.loc[usable_indices, x_index]
        y = df.loc[usable_indices, station]
        slope, intercept, _, _, _ = scipy.stats.linregress(x, y)

        for start_date, consecutive_day in missing_data.loc[station].items():
            end_date = start_date + consecutive_day
            day = start_date
            while day < end_date:
                data_nans_removed.loc[day, station] = intercept + slope * data_nans_removed.loc[day, x_index]
                day = day + pd.Timedelta(1, 'd')
    return data_nans_removed



In [52]:
def find_best_correlations(df, md, lt, check_only_complete):
    to_fill = pd.DataFrame(columns=['x', 'usable'])

    for station, missing in md.items():
        if missing == {}:
            continue

        correlations = pd.DataFrame(columns=['correlation', 'p_value', 'usable'], index=[])

        for col in df.columns:
            if col == station or (check_only_complete and any((lt[station] & lt[col]))):
                continue
            data_for_regression = df.loc[df.index[~(lt[station] | lt[col])]][[station, col]]
            corr, p_value = scipy.stats.pearsonr(data_for_regression[col], data_for_regression[station])
            df2 = pd.DataFrame([[corr, p_value, data_for_regression.index]],
                               columns=['correlation', 'p_value', 'usable'], index=[col])
            correlations = pd.concat([correlations, df2])

        st_max_corr = correlations['correlation'].idxmax()

        if correlations['p_value'].loc[st_max_corr] >= 0.05:
            print("P-Value is too large")

        df3 = pd.DataFrame([[st_max_corr, correlations.loc[st_max_corr, 'usable']]], columns=['x', 'usable'],
                           index=[station])
        to_fill = pd.concat([to_fill, df3])
    return to_fill


In [53]:
def fill_data(df):
    md = df.apply(consecutive_nans, 0)
    lt = df.isna()

    tf = find_best_correlations(df, md, lt, False)

    df_no_nans = calculate_missing_values(df, tf, md)

    md = df_no_nans.apply(consecutive_nans, 0)
    lt = df_no_nans.isna()

    tf = find_best_correlations(df_no_nans, md, lt, True)

    df_no_nans = calculate_missing_values(df_no_nans, tf, md)
    return df_no_nans

In [57]:
data = read_in_csv()

data_training = data.loc[data.index <= '2005']
data_validation = data.loc[data.index > '2005']



In [60]:
data_training_filled = fill_data(data_training)

,1515,1516,1518,1521,1719,1720,1722,1723,2040,2046,...,2751,2545,2740,210896,744612,744616,744618,744624,210888,210900
Date,,,,,,,,,,,,,,,,,,,,,
1950-12-31,-114.0,-8.0,-88.0,130.0,238.5,231.995463,204.5,194.5,305.0,408,...,408.160551,18.0,77.490733,362.0,116.428971,191.827112,203.048181,146.523739,438.0,414.0
1951-01-01,-122.0,-22.0,-102.0,110.0,200.5,198.151138,182.5,160.5,260.0,376,...,370.673263,20.0,63.335961,349.0,108.782307,184.488641,194.801046,134.498247,417.0,400.0
1951-01-02,-129.0,-32.0,-119.0,80.0,164.5,166.088094,164.5,114.5,220.0,334,...,349.251956,16.0,64.622758,328.0,101.723849,177.714668,187.188306,123.397794,390.0,388.0
1951-01-03,-134.0,-36.0,-124.0,60.0,132.5,137.587610,128.5,88.5,182.0,286,...,326.759583,16.0,61.405765,302.0,95.841800,172.069691,180.844356,114.147416,360.0,370.0
1951-01-04,-153.0,-52.0,-135.0,50.0,110.5,117.993528,108.5,80.5,147.0,239,...,308.551472,6.0,65.909556,270.0,90.547955,166.989211,175.134801,105.822075,322.0,349.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-12-28,58.0,38.0,-132.0,304.0,487.0,464.000000,155.0,226.0,453.0,88,...,254.000000,187.0,162.000000,303.0,-48.000000,140.000000,164.000000,70.670639,270.0,213.0
2004-12-29,101.0,172.0,68.0,380.0,524.0,466.000000,233.0,238.0,466.0,127,...,256.000000,216.0,177.000000,295.0,-44.000000,140.000000,167.000000,83.621168,261.0,214.0
2004-12-30,40.0,150.0,90.0,423.0,529.0,461.000000,298.0,244.0,495.0,224,...,260.000000,262.0,172.000000,295.0,-47.000000,130.000000,159.000000,86.396281,274.0,228.0


In [64]:
data_validation_filled = fill_data(data_validation)

In [63]:
stations_to_remove = ['744620',
'744622',
'2747',
'2740',
'210896',
'744612',
'744616',
'744618']

data_validation = data_validation.drop(stations_to_remove,axis=1)
data_training_filled = data_training_filled.drop(stations_to_remove,axis=1)


In [66]:
(data_validation_filled.isna().sum() == 0).sum()

48

In [67]:
(data_training_filled.isna().sum() == 0).sum()

48

In [68]:
data_validation_filled.to_csv('../data/data_validation.csv')
data_training_filled.to_csv('../data/data_training.csv')